<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Automation/blob/main/PreDocWebsiteScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [454]:
# ! pip install beautifulsoup4 requests

# Web scrap of the following page:

https://predoc.org/ra-opportunities

In [455]:
# Due to the characteristics in the page
# I used text extraction with index for the information
# maybe, using text[index:] can be a better way to extract information I guess

In [456]:
from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/ra-opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [457]:
div_tags = soup.find_all("div", class_ = "body-copy")

h_links = []

for div_tag in div_tags:
  li_tags = div_tag.find_all("li")
  for li_tag in li_tags:
    a_tag = li_tag.find("a")
    h_links.append(a_tag["href"])

h_links

['https://bit.ly/3LFuCod',
 'https://bit.ly/3PXATOv',
 'https://bit.ly/46fL6eX',
 'https://bit.ly/3PY7DHB',
 'https://bit.ly/3LmEvqJ',
 'https://bit.ly/3Csqpzn',
 'https://bit.ly/3OifEXv',
 'https://bit.ly/3W49Xya',
 'https://bit.ly/447jdE2',
 'https://twitter.com/GennaioliC/status/1673968653962735619?s=20',
 'https://bit.ly/3CxaaB0']

In [458]:
institutions = []
fields = []
deadlines = []
researchers =[]

div_tags = soup.find_all("div", class_ = "body-copy")

for div_tag in div_tags:
  li_tags = div_tag.find_all("li")
  for li_tag in li_tags:

    temp_text = li_tag.get_text()

    temp_inst = temp_text[temp_text.find("Institution"):].split(":")[1].split("\n")[0].strip()
    institutions.append(temp_inst)

    temp_field = temp_text[temp_text.find("Fields"):].split(":")[1].split("\n")[0].strip().replace(".", "")
    fields.append(temp_field)

    temp_deadline = temp_text[temp_text.find("Deadline"):].split(":")[1].split("\n")[0].strip()
    deadlines.append(temp_deadline)

    temp_researcher = temp_text[temp_text.find("Researcher"):].split("\n")[0].split(":")[-1].strip()
    if len(temp_researcher)<3:
      temp_researcher = ""
    researchers.append(temp_researcher)



In [459]:
print(len(institutions), len(h_links))

11 11


In [460]:
# make it as a dataframe

import pandas as pd

df_ra = pd.DataFrame({"Deadlines": deadlines,
                      "Institution":institutions,
                      "Researcher": researchers,
                      "Field":fields,
                      "Links":h_links})


# change the deadline into the consistent form
date_pattern = r"(?i)((?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2})"

df_ra['Deadline'] = df_ra['Deadlines'].str.extract(date_pattern, expand=False)
df_ra.loc[df_ra['Deadline'].isna() & df_ra['Deadlines'].str.contains(r'(?i)rolling'), 'Deadline'] = 'Rolling'
df_ra.loc[df_ra['Deadline'].isna() & df_ra['Deadlines'].str.contains(r'(?i)ASAP'), 'Deadline'] = 'ASAP'

In [461]:
df_ra.sample(3)

,Deadlines,Institution,Researcher,Field,Links,Deadline
3,"October 16, 2023",African Development Foundation,,Finance and budget analyst functions,https://bit.ly/3PY7DHB,October 16
4,"October 16, 2023",The U.S. Government Accountability Office (GAO),,Financial audits and analyses,https://bit.ly/3LmEvqJ,October 16
2,"October 10, 2023",The Federal Energy Regulatory Commission (FERC),,"Economics, environmental, energy industry, ele...",https://bit.ly/46fL6eX,October 10


# Web scrap of the following page
https://predoc.org/opportunities

In [462]:
from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [463]:
institutions = []
fields = []
deadlines = []
researchers =[]

copy_divs = soup.find_all('div', class_='copy')

for copy_div in copy_divs:
  temp_text = copy_div.get_text()

  temp_researcher = temp_text[temp_text.find("Researcher"):].split("\n")[0].split(":")[-1].strip()
  researchers.append(temp_researcher)

  temp_field = temp_text[temp_text.find("Field"):].split("\n")[0].split(":")[-1].strip()
  if temp_field.lower() == "field":
    temp_field = ""
  fields.append(temp_field)

  temp_institution = temp_text[temp_text.find("Institution"):].split("\n")[0].split(":")[-1].strip()
  institutions.append(temp_institution)

  temp_dead = temp_text[temp_text.find("Deadline"):].split("\n")[0]
  temp_dead_split = temp_dead.split(":")
  if len(temp_dead_split)>1:
    deadlines.append(temp_dead_split[1])
  else:
    deadlines.append("")


In [464]:
copy_divs = soup.find_all('div', class_='copy')
h_links = []

for copy_div in copy_divs:
  a_tag = copy_div.find("a")
  h_links.append(a_tag["href"])

In [465]:
import pandas as pd

df_op = pd.DataFrame({"Deadlines": deadlines,
                      "Institution":institutions,
                      "Researcher": researchers,
                      "Field":fields,
                      "Links":h_links})

df_op[df_op["Researcher"] == "Luigi Zingales"]

,Deadlines,Institution,Researcher,Field,Links
8,"Initial review on October 16, rolling thereafter",The Stigler Center for the Study of the Econom...,Luigi Zingales,"Finance, Microeconomics, Political Economy, Pu...",https://bit.ly/3lKJlQT
125,,University of Chicago Booth School of Business...,Luigi Zingales,"Economics, Political Economy, Finance, Public ...",https://bit.ly/3lKJlQT


In [466]:
import numpy as np

# change the deadline into the consistent form
date_pattern = r"(?i)((?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2})"

df_op['Deadline'] = df_op['Deadlines'].str.extract(date_pattern, expand=False)
df_op.loc[df_op['Deadline'].isna() & df_op['Deadlines'].str.contains(r'(?i)rolling'), 'Deadline'] = 'Rolling'
df_op.loc[df_op['Deadline'].isna() & df_op['Deadlines'].str.contains(r'(?i)ASAP'), 'Deadline'] = 'ASAP'

In [467]:
df_op = df_op[(df_op["Deadlines"] != "")]
df_op = df_op[(df_op["Deadline"] != "")]
df_op = df_op.dropna(subset="Deadline")

# Sort the data

In [468]:
import datetime

def convert_deadline(deadline):
  if deadline == "ASAP":
    return pd.to_datetime("2099-12-31")
  elif deadline == "Rolling":
    return pd.to_datetime("2098-12-31")
  elif pd.isna(deadline):
    return pd.to_datetime("2099-12-31")
  else:
    return pd.to_datetime(f"{datetime.date.today().year} {deadline}", format = "%Y %B %d")

df_ra["deadline_index"] = df_ra["Deadline"].apply(convert_deadline)

today = datetime.datetime.now().date()
df_ra = df_ra[df_ra["deadline_index"] >= pd.to_datetime(today)]

# sort the data by date
df_ra = df_ra.sort_values(by = "deadline_index", ascending=True)

# change the order
df_ra = df_ra[["Deadline", "Institution", "Researcher", "Field", "Links", "Deadlines"]]


In [469]:
df_op["deadline_index"] = df_op["Deadline"].apply(convert_deadline)


today = datetime.datetime.now().date()
df_op = df_op[df_op["deadline_index"] >= pd.to_datetime(today)]

# sort the data by date
df_op = df_op.sort_values(by = "deadline_index", ascending=True)

# change the order
df_op = df_op[["Deadline", "Institution", "Researcher", "Field", "Links", "Deadlines"]]

# Save the output as Exce lfile

In [470]:
import pandas as pd

with pd.ExcelWriter("output.xlsx") as writer:

    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_op.to_excel(writer, sheet_name="Opportunities page", index=False)
    df_ra.to_excel(writer, sheet_name="RA Opportunities page", index=False)


In [471]:
from google.colab import files
files.download("output.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Extra Codes
Opportunities page (with html header)

In [472]:
# In case of using html headers

from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [473]:
# All the values can be single or multiples
# The text data are from div (class = "copy")
# I assumed that the information posted in opportunities are aligned in the same format
# I got the hyperlinks from div.copy a.general-link

# the code for the link needs to be corrected

copy_divs = soup.find_all('div', class_='copy')

institutions = []
for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          if "Institution" in strong_tag.get_text():
            em_tag = strong_tag.find_next("em")
            institutions.append(em_tag.get_text())
            #print(em_tag.get_text())

In [474]:
# Deadlines
# nested under div > p > strong > Deadline

copy_divs = soup.find_all('div', class_='copy')

deadlines = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          # print(strong_tag.get_text(), strong_tag.next_sibling, "\n")
          if "Deadline" in strong_tag.get_text():
            # print(strong_tag.next_sibling)
            next_sibling = strong_tag.next_sibling
            if next_sibling:
              deadlines.append(next_sibling.strip())
            else:
              deadlines.append("")  # Add an empty string if "researchers" is not found

fields = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          # print(strong_tag.get_text(), strong_tag.next_sibling, "\n")
          if "Field" in strong_tag.get_text():
            # print(strong_tag.next_sibling)
            next_sibling = strong_tag.next_sibling
            if next_sibling:
              fields.append(next_sibling.strip().strip(":"))
            else:
              fields.append("")  # Add an empty string if "researchers" is not found


print(len(fields))


researchers = []
for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
      #print(p_tag.get_text(), "\n")
      if "Researcher" in p_tag.get_text() or "Researcher(s)" in p_tag.get_text():
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          if "Researcher" in strong_tag.get_text() or "Researcher(s)" in strong_tag.get_text() :
              next_sibling = strong_tag.next_sibling
              researchers.append(next_sibling.strip().strip(":").strip("()"))
      else:
        researchers.append("None")


links = []
a_tags = soup.select('div.copy a.general-link')
for a_tag in a_tags:
    # print(a_tag["href"])
    links.append(a_tag["href"])


58
